Bpmn2constraints is made to compile BPMN models from both `JSON` and `XML`, but since `JSON` models are proprietary to SAP Signavio, only `XML` support have been added in this repository. Go to [the original repository](https://github.com/signavio/bpmn2constraints) for the full version. 

In order to start using the tool, import the necessary packages.

In [1]:
import os
from Declare4Py.ProcessModels.BpmnConstraintsModel import BpmnConstraintsModel

/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


Some examples are provided in this repository, such as this linear sequence: 
<figure>
    <img width="900" height="200" src="data/linear_diagram.png">
</figure>

With bpmn2constraints, this diagram can be translated into DECLARE constraints, such as:

In [2]:
path = os.path.join("data","linear_sequence.xml")
model = BpmnConstraintsModel()
model.parse_from_file(path)
declare_model = model.declare_model
constraints = declare_model.get_decl_model_constraints()
activities = declare_model.get_model_activities()
print("Model activities:")
print("-----------------")
for idx, act in enumerate(activities):
    print(idx, act)
print("\n")
print("Model constraints:")
print("-----------------")
for idx, con in enumerate(constraints):
    print(idx, con)



Model activities:
-----------------


Model constraints:
-----------------


The BpmnConstraintsModel class contains constraints as LTLf as well.

The LTLf formulas are stored in a class containing:
```formula```, ```parsed_formula``` and ```is_satisfiable```

Or the result can be easily printed such as:

In [3]:
for ltl_mod in (model.ltl_model):
    print(ltl_mod.formula)
    print(ltl_mod.parsed_formula)
    print(f'Is satisfiable? {ltl_mod.check_satisfiability()}\n')


The following blocks are bigger and more complex examples of usage of bpmn2constraints. Based on a bpmn diagram from the Sepsis Cases event log.

<figure>
    <img width="700" height="1000" src="data/Sepsis Cases.png">
</figure>

In [4]:
path = "data/Sepsis Cases.xml" #Path to data 
model = BpmnConstraintsModel()
model.parse_from_file(model_path=path)   

declare_model = model.declare_model
constraints = declare_model.get_decl_model_constraints()
activities = declare_model.get_model_activities()

print("Model activities:")
print("-----------------")
for idx, act in enumerate(activities):
    print(idx, act)
print("\n")
print("Model constraints:")
print("-----------------")
for idx, con in enumerate(constraints):
    print(idx, con)

KeyboardInterrupt: 

In [ ]:

for ltl_mod in model.ltl_model:
    print(ltl_mod.formula)
    print(ltl_mod.parsed_formula)
    print(f'Is satisfiable? {ltl_mod.check_satisfiability()}\n')


con_erregistration
con_erregistration
Is satisfiable? True

(G((con_ertriage) ->  (F(con_ersepsistriage)))) & ((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)) | (G(~(con_ertriage))))
(and (always (implies con_ertriage (eventually con_ersepsistriage))) (or (eventually con_ersepsistriage) (and (not con_ertriage) con_ersepsistriage) (always (not con_ertriage))))
Is satisfiable? True

(F(con_ertriage)) & (F(con_ersepsistriage))
(and (eventually con_ertriage) (eventually con_ersepsistriage))
Is satisfiable? True

(G((con_ertriage) ->  (X[!]((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)))))) & (G((con_ersepsistriage) ->  ((X[!](~(con_ersepsistriage))) & ((F(con_ertriage)) | ((~(con_ersepsistriage)) & (con_ertriage))))))
(and (always (implies con_ertriage (next (or (eventually con_ersepsistriage) (and (not con_ertriage) con_ersepsistriage))))) (always (implies con_ersepsistriage (and (next (not con_ersepsistriage)) (or (eventually con_ertriage) (and 

Now that the BpmnConstraintsModel is set up, it can be used to check conformance

In [ ]:
from Declare4Py.D4PyEventLog import D4PyEventLog
from Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer

log_path = os.path.join("../../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

analyzer = LTLAnalyzer(event_log, model.ltl_model)
conf_check_res_df = analyzer.run_multiple_models(minimize_automaton=False) #This does not seem to work as intented currently
print(f"Accepted traces: {len(conf_check_res_df[conf_check_res_df['accepted'] == True])}")
print(conf_check_res_df)

for idx, ltl_mod in enumerate(model.ltl_model):
    analyzer = LTLAnalyzer(event_log, ltl_mod)
    conf_check_res_df = analyzer.run()
    print(ltl_mod.formula)
    print(f"{idx}. Accepted traces: {len(conf_check_res_df[conf_check_res_df['accepted'] == True])}")



/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|██████████| 1050/1050 [00:00<00:00, 3627.00it/s]
/home/fleph/sap/Declare4Py/.venv/lib64/python3.11/site-packages/pm4py/utils.py:486: UserWarning: the EventLog class has been deprecated and will be removed in a future release.
  warnings.warn("the EventLog class has been deprecated and will be removed in a future release.")


Accepted traces: 0
     case:concept:name  accepted
0                    A     False
1                    B     False
2                    C     False
3                    D     False
4                    E     False
...                ...       ...
1045               HNA     False
1046               INA     False
1047               JNA     False
1048               KNA     False
1049               LNA     False

[1050 rows x 2 columns]
con_erregistration
0. Accepted traces: 995
(G((con_ertriage) ->  (F(con_ersepsistriage)))) & ((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)) | (G(~(con_ertriage))))
1. Accepted traces: 1029
(F(con_ertriage)) & (F(con_ersepsistriage))
2. Accepted traces: 1049
(G((con_ertriage) ->  (X[!]((F(con_ersepsistriage)) | ((~(con_ertriage)) & (con_ersepsistriage)))))) & (G((con_ersepsistriage) ->  ((X[!](~(con_ersepsistriage))) & ((F(con_ertriage)) | ((~(con_ersepsistriage)) & (con_ertriage))))))
3. Accepted traces: 0
F((con_releasea) & (X[!](